In [ ]:
from spectralmc.sobol_sampler import SobolSampler, BoundSpec
from spectralmc.gbm import SimulationParams, BlackScholes
import pandas as pd

In [ ]:
sp = SimulationParams(
    timesteps=1024,
    network_size=256,
    batches_per_mc_run=4096,
    threads_per_block=256,
    mc_seed=42,
    buffer_size=1,
)

In [ ]:
total_size_mb=sp.memory_footprint_bytes()/1024/1024
print(total_size_mb)

In [ ]:
bs_dimensions = {
    "X0": BoundSpec(lower=0.001, upper=10000),
    "K":  BoundSpec(lower=0.001, upper=20000),
    "T":  BoundSpec(lower=0,     upper=10),
    "r":  BoundSpec(lower=-0.2,  upper=0.2),
    "d":  BoundSpec(lower=-0.2,  upper=0.2),
    "v":  BoundSpec(lower=0,     upper=2.0),
}

In [ ]:
ss = SobolSampler(pydantic_class=BlackScholes.Inputs,dimensions=bs_dimensions,seed=43)

In [ ]:
bs = BlackScholes(sp=sp)

In [ ]:
%%time
samples=ss.sample(2048)

In [ ]:
%%time
prices=[bs.price_to_host(input) for input in samples]

In [ ]:
bs._normal_gen.get_time_spent_synchronizing()